In [137]:
from service.technical_indicators import TechnicalIndicators
from bs4 import BeautifulSoup

In [126]:
import requests
import json

In [156]:
header = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    }
website = "https://www.tradingview.com/symbols/EURUSD/news/"
url_prefix = "https://r.jina.ai/"
website_url = url_prefix + website

response = requests.get(website_url)

In [141]:
html = response.text
soup = BeautifulSoup(html, parser="html.parser")

In [157]:
scrapped_content = response.text

In [158]:
print(scrapped_content)

Title: EURUSD Latest News and Headlines — TradingView

URL Source: https://www.tradingview.com/symbols/EURUSD/news/

Markdown Content:
TimeSymbolHeadlineProvider

[Euro falls after French exit poll points to hung parliament](https://www.tradingview.com/news/reuters.com,2024:newsml_L8N3IZ08V:0-euro-falls-after-french-exit-poll-points-to-hung-parliament/)[Euro Q3 Technical Forecast: A Mixed Picture for the Single Currency](https://www.dailyfx.com/analysis/euro-q3-technical-forecast-a-mixed-picture-for-the-single-currency-20240707.html)[Forexlive Americas FX news wrap 5 Jul: NFP for June sent the USD lower/yields tumbling](https://www.tradingview.com/news/forexlive:57acae61a094b:0-forexlive-americas-fx-news-wrap-5-jul-nfp-for-june-sent-the-usd-lower-yields-tumbling/)[US Dollar Q3 Technical Outlook – Bearish Forces to Prevail in the End](https://www.dailyfx.com/analysis/us-dollar-q3-technical-outlook-bearish-forces-to-prevail-in-the-end-20240705.html)[World stocks rise, US yields dip amid 

In [151]:
from config import Config
model = Config(model_name="gpt-4o").get_model()

In [152]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

class ArticlesExtraction(BaseModel):
    """Articles extraction output format"""

    article: str = Field(description="The name of the article")
    url: str = Field(description="The url of the article")

parser = PydanticOutputParser(pydantic_object=ArticlesExtraction)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are provided with the content scrapped from a website. Your task is to extract the first 5 appearing articles and their urls to the user. Wrap the output in `json` tags\n{format_instructions}"),
    ("user", "Below is the content. \n{content}")
]).partial(format_instructions=parser.get_format_instructions())

chain = prompt | model | StrOutputParser()

response = chain.invoke({"content": scrapped_content}).strip("```json").strip("```").strip()

In [153]:
websites = json.loads(response)

In [155]:
websites

[{'article': 'Euro falls after French exit poll points to hung parliament',
  'url': 'https://www.tradingview.com/news/reuters.com,2024:newsml_L8N3IZ08V:0-euro-falls-after-french-exit-poll-points-to-hung-parliament/'},
 {'article': 'Euro Q3 Technical Forecast: A Mixed Picture for the Single Currency',
  'url': 'https://www.dailyfx.com/analysis/euro-q3-technical-forecast-a-mixed-picture-for-the-single-currency-20240707.html'},
 {'article': 'Forexlive Americas FX news wrap 5 Jul: NFP for June sent the USD lower/yields tumbling',
  'url': 'https://www.tradingview.com/news/forexlive:57acae61a094b:0-forexlive-americas-fx-news-wrap-5-jul-nfp-for-june-sent-the-usd-lower-yields-tumbling/'},
 {'article': 'US Dollar Q3 Technical Outlook – Bearish Forces to Prevail in the End',
  'url': 'https://www.dailyfx.com/analysis/us-dollar-q3-technical-outlook-bearish-forces-to-prevail-in-the-end-20240705.html'},
 {'article': 'World stocks rise, US yields dip amid jobs data, UK labor landslide',
  'url': '

In [86]:
contents = []
for website in websites:
    response = requests.get(url_prefix+website["url"])
    if response.status_code == 200:
        content = response.text
        contents.append(content)


In [88]:
print(contents[1])

Title: Forexlive Americas FX news wrap 5 Jul: NFP for June sent the USD lower/yields tumbling

URL Source: https://www.tradingview.com/news/forexlive:57acae61a094b:0-forexlive-americas-fx-news-wrap-5-jul-nfp-for-june-sent-the-usd-lower-yields-tumbling/

Published Time: 2024-07-05T21:37:51+00:00

Markdown Content:
Forexlive Americas FX news wrap 5 Jul: NFP for June sent the USD lower/yields tumbling — TradingView News
                                                                                                                           

 

[](https://www.tradingview.com/)

Search

*   [Products](https://www.tradingview.com/chart/)
*   [Community](https://www.tradingview.com/community/)
*   [Markets](https://www.tradingview.com/markets/)
*   [News](https://www.tradingview.com/news/)
*   [Brokers](https://www.tradingview.com/brokers/)
*   [More](https://www.tradingview.com/support/)

EN ![Image 1](https://www.tradingview.com/news/forexlive:57acae61a094b:0-forexlive-americas-fx-news-wr

In [119]:
model = Config(model_name="gpt-3.5-turbo-0125", temperature=0).get_model()

In [120]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are provided with the content scrapped from a website. Your task is to summarize the key information about EUR USD."),
    ("user", "{content}")
])

chain = prompt | model | StrOutputParser()

results = chain.batch([{"content": content} for content in contents])

In [121]:
for result in results:
    print(result, "\n")

The Euro Q3 Technical Forecast suggests a mixed picture for the EUR/USD pair. The Euro has exhibited a choppy downtrend in 2024, reacting to rate cut projections in the US and ECB. The pair may see a recovery in Q3 but remains uncertain due to French election concerns and Fed easing policy. The technical analysis indicates key resistance levels at 1.0929 and 1.1033, while support levels are at 1.0600 and 1.0516. The Euro may face challenges in gaining strength compared to other G7 currencies. Additionally, the Bank of England's potential rate cut could impact the EUR/GBP pair, with levels to watch at 0.8472 and 0.8720. The overall outlook suggests a recovery for EUR/GBP but allows for a possible initial move lower in Q3. 

Key Information about EUR/USD from the provided content:

- The EUR/USD pair was mentioned in the context of Forex trading news on July 5, 2024.
- The US dollar weakened after the Non-Farm Payroll (NFP) report for June, causing yields to tumble.
- The NASDAQ closed a

In [174]:
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("ALPHA_VANTAGE_API_KEY")

In [178]:
url = f"https://www.alphavantage.co/query?function=CURRENCY_EXCHANGE_RATE&from_currency=EUR&to_currency=USD&apikey={api_key}"
r = requests.get(url)
data = r.json()

print(data)

{'Realtime Currency Exchange Rate': {'1. From_Currency Code': 'EUR', '2. From_Currency Name': 'Euro', '3. To_Currency Code': 'USD', '4. To_Currency Name': 'United States Dollar', '5. Exchange Rate': '1.08190000', '6. Last Refreshed': '2024-07-07 21:30:34', '7. Time Zone': 'UTC', '8. Bid Price': '1.08186000', '9. Ask Price': '1.08195000'}}


In [179]:
url = f"https://www.alphavantage.co/query?function=FX_DAILY&from_symbol=EUR&to_symbol=USD&apikey={api_key}"
r = requests.get(url)
data = r.json()

In [180]:
print(data)

{'Meta Data': {'1. Information': 'Forex Daily Prices (open, high, low, close)', '2. From Symbol': 'EUR', '3. To Symbol': 'USD', '4. Output Size': 'Compact', '5. Last Refreshed': '2024-07-05 21:55:00', '6. Time Zone': 'UTC'}, 'Time Series FX (Daily)': {'2024-07-05': {'1. open': '1.08099', '2. high': '1.08426', '3. low': '1.07968', '4. close': '1.08370'}, '2024-07-03': {'1. open': '1.07933', '2. high': '1.08167', '3. low': '1.07842', '4. close': '1.07933'}, '2024-07-02': {'1. open': '1.07484', '2. high': '1.08156', '3. low': '1.07371', '4. close': '1.07484'}, '2024-07-01': {'1. open': '1.07371', '2. high': '1.07461', '3. low': '1.07106', '4. close': '1.07371'}, '2024-06-30': {'1. open': '1.07353', '2. high': '1.07782', '3. low': '1.07208', '4. close': '1.07353'}, '2024-06-27': {'1. open': '1.07085', '2. high': '1.07210', '3. low': '1.06860', '4. close': '1.07085'}, '2024-06-26': {'1. open': '1.06801', '2. high': '1.07262', '3. low': '1.06778', '4. close': '1.06801'}, '2024-06-25': {'1. o